In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# **Importing nessecary libraries**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score,GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data importing and preprocessing

In [3]:
traindf1=pd.read_csv('/kaggle/input/titanic/train.csv')
traindf1.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
traindf1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#creating a new dataframe to edit without loosing orignal
traindf2=traindf1

In [6]:
#dropping columns which are not required in training
traindf2.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

In [7]:
#checking for null values in each column
traindf2.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [8]:
#also dropping Cabin because out of around 900 values 700 are null so can't manage that much null values
traindf2.drop('Cabin',axis=1,inplace=True)

In [9]:
# filling all null values in age by the mean age
traindf2.Age.fillna(int(traindf2.Age.mean()),inplace=True)

In [10]:
#checking for other columns with null values
traindf2.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [11]:
#only two null values in embarked and can't think of a desirable way of filling them so droping the rows with null values
traindf2.dropna(inplace=True)

In [12]:
#printing the dataframe to look at the values
traindf2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [13]:
#using label encoder to convert text into numbers
le=LabelEncoder()
traindf2.Sex=le.fit_transform(traindf2.Sex)
traindf2.Embarked=le.fit_transform(traindf2.Embarked)

In [14]:
#printing the dataframe to look at the values
traindf2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [15]:
#making sure no values are null
traindf2.isnull().any()

Survived    False
Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [16]:
y=traindf2.Survived

In [17]:
x=traindf2.drop('Survived',axis=1)
x.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# **Data preprocessing for the training dataset done**

# **Model building**  

In [19]:
svm=SVC()
# Define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(svm, param_grid, cv=5)

# Perform grid search with cross-validation
grid_search.fit(x_train, y_train)

# Get the best estimator from grid search
best_svm = grid_search.best_estimator_

# Access the best training accuracy during grid search
training_accuracy = grid_search.best_score_

#printing the best training accuracy during grid search
print("training score:",training_accuracy)
# Evaluate the best estimator on the test set
test_accuracy = best_svm.score(x_test, y_test)

# printing the test accuracy
print("Test accuracy:", test_accuracy)

training score: 0.7833940707180144
Test accuracy: 0.797752808988764


# *Even with the best params the accuracy is too low. trying another model i.e GaussianNB*

In [20]:
# Create a Gaussian Naive Bayes classifier
gnb = GaussianNB()

#training the model
gnb.fit(x_train,y_train)

#evaluating on unseen data
print(gnb.score(x_test,y_test))

0.797752808988764


the score is better than the previous one but will make another model i.e XGboost to see which is better

In [21]:
# Create an XGBoost classifier
xgb = XGBClassifier()

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 4, 5]
}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(xgb, param_grid, cv=5)

# Perform grid search with cross-validation
grid_search.fit(x_train, y_train)

# Get the best estimator from grid search
best_xgb = grid_search.best_estimator_

# Evaluate the best estimator on the test set
test_accuracy = best_xgb.score(x_test, y_test)

# Print the test accuracy
print("Test accuracy:", test_accuracy)

Test accuracy: 0.8202247191011236


this is more better
making final predictions using this model

# now to make prediction on actual test dataset

# *Importing and preprocessing the test data*

In [22]:
#importing two instances
testdf1=pd.read_csv('/kaggle/input/titanic/test.csv')
testdf2=pd.read_csv('/kaggle/input/titanic/test.csv')
testdf1.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
# doing the same steps as on the previous training data set 

# dropping irrelevant columns
testdf1.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

#label encoding 'Sex' and 'Embarked' columns
testdf1.Sex=le.fit_transform(testdf1.Sex)
testdf1.Embarked=le.fit_transform(testdf1.Embarked)

#Checking for any blank values
print(testdf1.isnull().sum())

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64


In [24]:
# filling null values
testdf1.Age.fillna(int(testdf1.Age.mean()),inplace=True)
testdf1.Fare.fillna(int(testdf1.Fare.mean()),inplace=True)

In [25]:
#again checking for any null values
testdf1.isnull().any()

Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [26]:
#looking at the dataset
testdf1.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


# *Finally predicting on the test data*

In [27]:
predictions=best_xgb.predict(testdf1)

In [28]:
#creating a dataframe from the predictions with respect to Passenger Id
data={'PassengerID':testdf2.PassengerId,
     'Survived':predictions}
xgbpredictionsfinal=pd.DataFrame(data)

#viewing the dataframe
xgbpredictionsfinal.head()

,PassengerID,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [29]:
#exporting a csv file of the predictions
xgbpredictionsfinal.to_csv( ' xgbpredictionsfinal.csv',index=False)